## Model Deployment Using FastAPI

This section describes the deployment of our trained machine learning model through a RESTful API created with FastAPI. The API fetches the optimal model from the MLflow artifact repository and offers a (`/predict`) endpoint. This endpoint receives input features in JSON format and outputs the corresponding prediction.

In [ ]:
# import necessary libraries
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import mlflow.pyfunc
import pandas as pd
import numpy as np

# Initialize FastAPI app
app = FastAPI(
    title="German Credit Model API",
    description="API for predicting credit risk",
    version="1.0.0"
)

# Define the expected input schema using Pydantic
class InputData(BaseModel):
    ExistingAccount_Status: str
    Duration_Months: int
    Credit_History: str
    Purpose: str
    Credit_Amount: float
    SavingsAccount_Bonds: str
    Present_Employment_Since: str
    Installment_Rate: int
    PersonalStatus_Sex: str
    OtherDebtors_Guarantors: str
    Present_Residence_Since: int
    Property: str
    Age_Years: int
    Other_Installment_Plans: str
    Housing: str
    Num_Existing_Credits: int
    Job: str
    Num_People_Liable: int
    Telephone: str
    Foreign_Worker: str

# Load the best model directly from MLflow artifact location
model_path = "C:/Users/kunal/Downloads/MLOps/Project/Gr-05_MLOps_Project/mlruns/540899927217565002/5ffe5026e5ef47efa4269d28d2d86080/artifacts/model"

try:
    model = mlflow.pyfunc.load_model(model_path)
except Exception as e:
    raise RuntimeError(f"Failed to load the model: {e}")


@app.post("/predict", summary="Get Prediction", response_description="Prediction Result")
def predict(input_data: InputData):
    """
    Receives input data as JSON, processes it to make a prediction with a pre-trained model, and returns the prediction.

    Parameters:
    - input_data (InputData): Structured input data validated against a Pydantic model.

    Returns:
    - dict: A dictionary with the prediction results, formatted for JSON response.

    Exceptions:
    - HTTPException: Returns a 500 error if there are issues during processing.

    The function converts the JSON input to a DataFrame, performs a prediction, 
    converts the result to a Python-compatible type, and handles any exceptions.
    """
    try:
        # Convert input to DataFrame
        input_df = pd.DataFrame([input_data.dict()])
        
        # Make prediction
        prediction = model.predict(input_df)
        
        # Convert numpy types to native Python types for JSON serialization
        if isinstance(prediction, np.ndarray):
            prediction = prediction.tolist()
        elif isinstance(prediction, (np.float64, np.int64)):
            prediction = prediction.item()
        
        return {"prediction": prediction}
    except Exception as err:
        raise HTTPException(status_code=500, detail=str(err))
        